# Импорты

In [1]:
!pip install pymorphy2

In [2]:
!pip install python-rake

In [3]:
!pip install summa

In [4]:
import pandas as pd
import numpy as np
from string import punctuation
punctuation += '–—«»…'
from nltk.corpus import stopwords


import nltk
nltk.download('stopwords')

from pymorphy2 import MorphAnalyzer
from pymorphy2.tokenizers import simple_word_tokenize
m = MorphAnalyzer()

import RAKE
from summa import keywords
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Данные
Я выбрал статьи с Naked science в качестве данных. Всего 11 статей из разных категорий. 

In [5]:
df = pd.read_csv('naked_science4.csv')

In [6]:
df

,text,original_keywords,my_keywords
0,Исследователи из компании Valisure обнаружили ...,"бензол, дезодорант, канцерогены, рак, химикаты","дезодорант, бензол, канцерогены"
1,Международная группа нейробиологов обнаружила ...,"акупунктура, иглоукалывание, надпочечники, ней...","иглоукалывание ,надпочечники, нейроны"
2,Институт космического ведомства ЦНИИмаш заказа...,"двигатели, многоразовая ракета, Роскосмос","двигатели, ракета, роскосмос, ступень"
3,В составе твердых экзопланет доля железа может...,"астрономия, металличность, экзопланеты","экзопланеты, железо, звезда"
4,"Ученые предложили метод, с помощью которого др...","древесина, нержавеющая сталь, целлюлоза","древесина, сталь, целлюлоза, нож"
5,Российские ученые рассказали об одной из мощне...,"Земля, магнитная буря, солнечная вспышка, Солнце","Земля, Солнце, магнитная буря, магнитное поле,..."
6,Специалисты Красногорского завода им. С. А. Зв...,"Зенит, прицел, снайперская винтовка, стрелково...","прицел, винтовка, снайперская винттовка, дальн..."
7,"Во всяком случае, это единственное имеющееся о...","акулы, биология, ихтиология, Рыбы","акулы, рыбы, шкура, акулья кожа, трение"
8,Чтобы изучить строение тканей и органов живых ...,"гистология, клетки, окрашивание, пурпур, эозин","гистология, красители, клетки, окрашивание, ткани"
9,"Анализ ДНК грызуна, типичного для Скандинавии,...","археология, викинги, донные отложения, колониз...","мыши, фекалии, архипелаг, острова, викинги"


Пересечение слишком маленькое, решил взять оббъединение

In [7]:
[list(set(df.original_keywords[i].split(', ')) & set(df.my_keywords[i].split(', '))) for i in range(len(df))]

[['дезодорант', 'бензол'],
 ['нейроны'],
 ['двигатели'],
 ['экзопланеты'],
 ['древесина', 'целлюлоза'],
 ['Земля', 'Солнце', 'магнитная буря'],
 ['прицел'],
 ['акулы'],
 ['окрашивание', 'клетки', 'гистология'],
 ['фекалии', 'викинги', 'мыши'],
 ['Марс', 'метеориты', 'кратеры']]

In [8]:
union = [list(set(df.original_keywords[i].split(', ')) | set(df.my_keywords[i].split(', '))) for i in range(len(df))]
df['golden'] = [', '.join(k) for k in union]
union

[['канцерогены ', 'рак', 'химикаты', 'дезодорант', 'канцерогены', 'бензол'],
 ['иглоукалывание',
  'акупунктура',
  'надпочечники',
  'цитокиновый шторм',
  'нейроны',
  'иглоукалывание ,надпочечники'],
 ['ракета',
  'многоразовая ракета',
  'Роскосмос',
  'ступень ',
  'роскосмос',
  'двигатели'],
 ['астрономия', 'металличность', 'железо', 'экзопланеты', 'звезда'],
 ['древесина', 'нож', 'целлюлоза', 'сталь', 'нержавеющая сталь'],
 ['Земля',
  'Солнце',
  'вспышка',
  'магнитное поле',
  'солнечная вспышка',
  'солнечная активность',
  'магнитная буря'],
 ['винтовка',
  'снайперское оружие',
  'стрелковое оружие',
  'снайперская винттовка',
  'дальность',
  'прицел',
  'снайперская винтовка',
  'Зенит'],
 ['Рыбы',
  'акулы',
  'акулья кожа',
  'трение',
  'ихтиология',
  'рыбы',
  'шкура',
  'биология'],
 ['клетки',
  'пурпур',
  'красители',
  'гистология',
  'ткани',
  'окрашивание',
  'эозин'],
 ['фекалии',
  'мыши',
  'колонизация',
  'археология',
  'острова',
  'викинги',
  'архи

In [9]:
def normalize_text(text):
  for p in punctuation:
    text = text.replace(p, '')
  lemmas = []
  for t in simple_word_tokenize(text):
      lemmas.append(
          m.parse(t)[0].normal_form
      )
  return ' '.join(lemmas)

In [10]:
df['text'] = df['text'].apply(normalize_text)

In [11]:
df

,text,original_keywords,my_keywords,golden
0,исследователь из компания valisure обнаружить ...,"бензол, дезодорант, канцерогены, рак, химикаты","дезодорант, бензол, канцерогены","канцерогены , рак, химикаты, дезодорант, канце..."
1,международный группа нейробиолог обнаружить ме...,"акупунктура, иглоукалывание, надпочечники, ней...","иглоукалывание ,надпочечники, нейроны","иглоукалывание, акупунктура, надпочечники, цит..."
2,институт космический ведомство цниимаш заказат...,"двигатели, многоразовая ракета, Роскосмос","двигатели, ракета, роскосмос, ступень","ракета, многоразовая ракета, Роскосмос, ступен..."
3,в состав твёрдый экзопланета доля железо мочь ...,"астрономия, металличность, экзопланеты","экзопланеты, железо, звезда","астрономия, металличность, железо, экзопланеты..."
4,учёный предложить метод с помощь который древе...,"древесина, нержавеющая сталь, целлюлоза","древесина, сталь, целлюлоза, нож","древесина, нож, целлюлоза, сталь, нержавеющая ..."
5,российский учёный рассказать о один из мощный ...,"Земля, магнитная буря, солнечная вспышка, Солнце","Земля, Солнце, магнитная буря, магнитное поле,...","Земля, Солнце, вспышка, магнитное поле, солнеч..."
6,специалист красногорский завод они с а зверев ...,"Зенит, прицел, снайперская винтовка, стрелково...","прицел, винтовка, снайперская винттовка, дальн...","винтовка, снайперское оружие, стрелковое оружи..."
7,в всякий случай это единственный иметься объяс...,"акулы, биология, ихтиология, Рыбы","акулы, рыбы, шкура, акулья кожа, трение","Рыбы, акулы, акулья кожа, трение, ихтиология, ..."
8,чтобы изучить строение ткань и орган живой орг...,"гистология, клетки, окрашивание, пурпур, эозин","гистология, красители, клетки, окрашивание, ткани","клетки, пурпур, красители, гистология, ткани, ..."
9,анализ днк грызун типичный для скандинавия пок...,"археология, викинги, донные отложения, колониз...","мыши, фекалии, архипелаг, острова, викинги","фекалии, мыши, колонизация, археология, остров..."


In [12]:
c = 0
for i in df['text'].to_list():
  c += len(i.split())
c

5461

Слово "это" решил убрать, так как мешается

In [13]:
stop = stopwords.words('russian')
stop.append('это')

# Методы
Я выбрал rake textrank и tf--idf

In [14]:
rake = RAKE.Rake(stop)

rake_kw = []
for i in range(len(df)):
  kw_line = []
  for kw in rake.run(df['text'][i], maxWords=3, minFrequency=1):
    kw_line.append(kw[0])
  rake_kw.append(', '.join(kw_line))

df['rake_kw'] = rake_kw

In [15]:
textrank_kw = []
for i in range(len(df)):
  kw_line = []
  for kw in keywords.keywords(df['text'][i], language='russian', additional_stopwords=stop, scores=True):
    kw_line.append(kw[0])
  textrank_kw.append(', '.join(kw_line))

df['textrank_kw'] = textrank_kw

In [16]:
vectorizer = TfidfVectorizer(stop_words=stop)

tfiidf_kw = []
for i in range(len(df)):
  kw_line = []
  X = vectorizer.fit_transform([df['text'][i]])
  feature_names = np.array(vectorizer.get_feature_names())
  words = np.array(X.todense()).argsort()[0]
  tfiidf_kw.append(', '.join(feature_names[words[::-1][:20]]))

df['tfidf_kw'] = tfiidf_kw

In [17]:
df

,text,original_keywords,my_keywords,golden,rake_kw,textrank_kw,tfidf_kw
0,исследователь из компания valisure обнаружить ...,"бензол, дезодорант, канцерогены, рак, химикаты","дезодорант, бензол, канцерогены","канцерогены , рак, химикаты, дезодорант, канце...","некоторый случай достигать, превышать норма fd...","высокий уровень бензол, valisure, партия, s, к...","бензол, ppm, уровень, партия, часть, valisure,..."
1,международный группа нейробиолог обнаружить ме...,"акупунктура, иглоукалывание, надпочечники, ней...","иглоукалывание ,надпочечники, нейроны","иглоукалывание, акупунктура, надпочечники, цит...","действенный правда лишь, определённый зона тел...","который, учёный, год, действие, работа, сенсор...","нейрон, который, задний, учёный, иглоукалывани..."
2,институт космический ведомство цниимаш заказат...,"двигатели, многоразовая ракета, Роскосмос","двигатели, ракета, роскосмос, ступень","ракета, многоразовая ракета, Роскосмос, ступен...","реактивный двигатель активироваться, готовый м...","демонстратор многоразовый ракета, крылатый сту...","многоразовый, ступень, ракета, год, крылатый, ..."
3,в состав твёрдый экзопланета доля железо мочь ...,"астрономия, металличность, экзопланеты","экзопланеты, железо, звезда","астрономия, металличность, железо, экзопланеты...","довольно давно подметить, целое однако исследо...","планета, звезда, экзопланета, железо мочь, авт...","состав, звезда, планета, железо, экзопланета, ..."
4,учёный предложить метод с помощь который древе...,"древесина, нержавеющая сталь, целлюлоза","древесина, сталь, целлюлоза, нож","древесина, нож, целлюлоза, сталь, нержавеющая ...","часто негативно воздействовать, придание желае...","древесина, нож, высокий, способ, предложить ме...","древесина, нож, острый, материал, предложить, ..."
5,российский учёный рассказать о один из мощный ...,"Земля, магнитная буря, солнечная вспышка, Солнце","Земля, Солнце, магнитная буря, магнитное поле,...","Земля, Солнце, вспышка, магнитное поле, солнеч...","мощный магнитный буря, полуночь скачок увеличи...","солнечный, вспышка, солнце, земля, магнитная б...","вспышка, земля, солнечный, максимальный, учёны..."
6,специалист красногорский завод они с а зверев ...,"Зенит, прицел, снайперская винтовка, стрелково...","прицел, винтовка, снайперская винттовка, дальн...","винтовка, снайперское оружие, стрелковое оружи...","специалист красногорский завод, торговый марка...","новый снайперский прицел, кмз, позволять, год,...","прицел, новый, снайперский, винтовка, свч, поз..."
7,в всякий случай это единственный иметься объяс...,"акулы, биология, ихтиология, Рыбы","акулы, рыбы, шкура, акулья кожа, трение","Рыбы, акулы, акулья кожа, трение, ихтиология, ...","единственный иметься объяснение, журнал ecolog...","акула, рыбий, свойство акулий кожа, вид рыба, ...","акула, рыба, тереться, зуб, кожа, вид, трение,..."
8,чтобы изучить строение ткань и орган живой орг...,"гистология, клетки, окрашивание, пурпур, эозин","гистология, красители, клетки, окрашивание, ткани","клетки, пурпур, красители, гистология, ткани, ...","изучить строение ткань, невозможно различить к...","гистолог, краситель гистология, клетка, препар...","ткань, гистолог, цвет, срез, краситель, ядро, ..."
9,анализ днк грызун типичный для скандинавия пок...,"археология, викинги, донные отложения, колониз...","мыши, фекалии, архипелаг, острова, викинги","фекалии, мыши, колонизация, археология, остров...","клад якобы обнаружить, карман местный житель, ...","мышь, викинг, стать, статья, место, который, ч...","остров, мышь, викинг, азорский, место, год, ар..."


# Шаблоны
Посмотрим на прил+сущ, сущ+сущ и суш+глагол

In [18]:
templates = [['NOUN'], ['ADJF', 'NOUN'], ['NOUN', 'NOUN'], ['NOUN', 'VERB']]

In [19]:
def template(keywords):
  all_pos = []
  output = []
  keywords = keywords.split(', ')
  for kw in keywords:
    pos = []
    for w in kw.split():
      pos.append(morph.parse(w)[0].tag.POS)
    all_pos.append((pos))
  for i, kw in enumerate(keywords):
    if all_pos[i] in templates:
      output.append(kw)
  return output

In [20]:
for i in ['rake_kw', 'textrank_kw', 'tfidf_kw']:
  df[f'{i}_pos'] = df[i].apply(template)

df['golden_pos'] = df['golden'].apply(template)

In [21]:
for i in ['original_keywords',	'my_keywords',	'golden', 'rake_kw',	'textrank_kw',	'tfidf_kw']:
  df[i] = df[i].apply(normalize_text).apply(str.split)

In [22]:
df

,text,original_keywords,my_keywords,golden,rake_kw,textrank_kw,tfidf_kw,rake_kw_pos,textrank_kw_pos,tfidf_kw_pos,golden_pos
0,исследователь из компания valisure обнаружить ...,"[бензол, дезодорант, канцероген, рак, химикат]","[дезодорант, бензол, канцероген]","[канцероген, рак, химикат, дезодорант, канцеро...","[некоторый, случай, достигать, превышать, норм...","[высокий, уровень, бензол, valisure, партия, s...","[бензол, ppm, уровень, партия, часть, valisure...","[бензол уровень, бутан пропан, большой время, ...","[партия, бренд, средство, риск, химический вещ...","[бензол, уровень, партия, часть, вещество, сре...","[канцерогены , рак, химикаты, дезодорант, канц..."
1,международный группа нейробиолог обнаружить ме...,"[акупунктура, иглоукалывание, надпочечник, ней...","[иглоукалывание, надпочечник, нейрон]","[иглоукалывание, акупунктура, надпочечник, цит...","[действенный, правда, лишь, определённый, зона...","[который, учёный, год, действие, работа, сенсо...","[нейрон, который, задний, учёный, иглоукалыван...","[район живот, фасция живот, иглоукалывание моч...","[учёный, год, действие, работа, сенсорный нейр...","[нейрон, учёный, иглоукалывание, электроакупун...","[иглоукалывание, акупунктура, надпочечники, ци..."
2,институт космический ведомство цниимаш заказат...,"[двигатель, многоразовый, ракета, роскосмос]","[двигатель, ракета, роскосмос, ступень]","[ракета, многоразовый, ракета, роскосмос, ступ...","[реактивный, двигатель, активироваться, готовы...","[демонстратор, многоразовый, ракета, крылатый,...","[многоразовый, ступень, ракета, год, крылатый,...","[будущий испытание, м мясищев, выход ракета, с...","[двигатель, год, создание, ведомство, крыло, в...","[ступень, ракета, год, двигатель, проект, демо...","[ракета, многоразовая ракета, Роскосмос, ступе..."
3,в состав твёрдый экзопланета доля железо мочь ...,"[астрономия, металличность, экзопланета]","[экзопланета, железо, звезда]","[астрономия, металличность, железо, экзопланет...","[довольно, давно, подметить, целое, однако, ис...","[планета, звезда, экзопланета, железо, мочь, а...","[состав, звезда, планета, железо, экзопланета,...","[чей спектр, потенциальный интересность, некот...","[планета, звезда, экзопланета, железо мочь, ав...","[состав, звезда, планета, железо, экзопланета,...","[астрономия, металличность, железо, экзопланет..."
4,учёный предложить метод с помощь который древе...,"[древесина, нержавеющий, сталь, целлюлоза]","[древесина, сталь, целлюлоза, нож]","[древесина, нож, целлюлоза, сталь, нержавеющий...","[часто, негативно, воздействовать, придание, ж...","[древесина, нож, высокий, способ, предложить, ...","[древесина, нож, острый, материал, предложить,...","[столовый нож, метод древесина, прочный учёный...","[древесина, нож, способ, этап, вода, целлюлоза...","[древесина, нож, материал, этап, метод, произв...","[древесина, нож, целлюлоза, сталь, нержавеющая..."
5,российский учёный рассказать о один из мощный ...,"[земля, магнитный, буря, солнечный, вспышка, с...","[земля, солнце, магнитный, буря, магнитный, по...","[земля, солнце, вспышка, магнитный, поле, солн...","[мощный, магнитный, буря, полуночь, скачок, ув...","[солнечный, вспышка, солнце, земля, магнитный,...","[вспышка, земля, солнечный, максимальный, учён...","[максимальный удар, час ночь, спокойный значен...","[вспышка, солнце, земля, магнитная буре, го, п...","[вспышка, земля, учёный, солнце, год, сила, бу...","[Земля, Солнце, вспышка, магнитное поле, солне..."
6,специалист красногорский завод они с а зверев ...,"[зенит, прицел, снайперский, винтовка, стрелко...","[прицел, винтовка, снайперский, винттовка, дал...","[винтовка, снайперский, оружие, стрелковый, ор...","[специалист, красногорский, завод, торговый, м...","[новый, снайперский, прицел, кмз, позволять, г...","[прицел, новый, снайперский, винтовка, свч, по...","[тактический стрельба, качество основа, издели...","[год, оружие, винтовка свч, калашников, марка,...","[прицел, винтовка, свч, марка, изделие, оружие...","[винтовка, снайперско

# Метрики


In [23]:
def evaluate(method):
  if method.endswith('_pos'):
    golden = df['golden_pos']
  else:
    golden = df['golden']

  predicted = df[method]

  precision = []
  recall = []
  f1_score = []

  for i in range(len(golden)):
    prec = np.intersect1d(golden[i], predicted[i]).shape[0] / len(predicted[i])
    rec = np.intersect1d(golden[i], predicted[i]).shape[0] / len(golden[i])
   
    if prec + rec != 0:
      f1 = 2 * prec * rec / (prec + rec)
    else:
      f1 = 0
    precision.append(prec)
    recall.append(rec)
    f1_score.append(f1)

  return np.mean(precision), np.mean(recall), np.mean(f1_score)

In [24]:
methods = 'rake_kw	textrank_kw	tfidf_kw	rake_kw_pos	textrank_kw_pos	tfidf_kw_pos'.split()
metrics = {}
for m in methods:
  metrics[m] = evaluate(m)
df_res = pd.DataFrame(metrics).T
df_res.columns = ['precision', 'recall', 'f1 score']

In [25]:
df_res

,precision,recall,f1 score
rake_kw,0.038540,0.481477,0.069744
textrank_kw,0.107999,0.539886,0.176973
tfidf_kw,0.195455,0.455323,0.268261
rake_kw_pos,0.022923,0.082576,0.032954
textrank_kw_pos,0.063598,0.147944,0.087599
tfidf_kw_pos,0.103628,0.200974,0.135436


# Анализ

Морфологические шаблоны снизили качество. Видимо, нужна доработка этих шаблонов и больше экспериментов. Также мне кажется сильно влияет специфика данных: в новосттях о науке часто используют разные слова и конструкции для описания одной и той же сущности, чтобы текст был читаем для человека. 

Лучше всего показал себя TF-IDF. Почему?
Мне кажется, что авторы новостей хорошо постарались над ключевыми словами. Они скорее используются для навигации по статьям, а TF-IDF использовался в первых поисковых алгоритмах, поэтому результат такой.
Хуже всего работает RAKE. 

Для улучшения я бы поработал над шаблонами. Также интересно было бы взять новости из одной тематики и смотреть, как они различаются/сходятся по ключевым словам. Вообще для новостей о науке нужно как-то выделять название организаций/структур/стран и имена ученых, сделать это на нескольких уровнях.

Ну и конечно я бы сделал что-нибудь на модных нынче эмбеддингах: например, смотреть расстояние между эмбеддингами наших слов и текстов.